<a href="https://colab.research.google.com/github/afortuny/DeepLearningFastAI/blob/main/13_language_model_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [97]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [98]:
#hide
from fastbook import *
from IPython.display import display,HTML

# Music model from scratch

In [99]:
!pip install music21
!apt-get install -y lilypond

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
lilypond is already the newest version (2.18.2-12build1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [100]:
#Importing Libraries
import numpy as np 
import pandas as pd 
from collections import Counter
import random
import IPython
from IPython.display import Image, Audio
import music21
from music21 import *
import matplotlib.pyplot as plt 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline
import sys
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")
np.random.seed(42)

In [101]:
#Loading the list of chopin's midi files as stream 
filepath = "/content/gdrive/MyDrive/Music/bach"
#Getting midi files
all_midis= []
for i in os.listdir(filepath):
    if i.endswith(".mid"):
        tr = filepath+'/'+i
        midi = converter.parse(tr)
        all_midis.append(midi)

In [102]:
all_midis

[<music21.stream.Score 0x7fdf639f8fd0>,
 <music21.stream.Score 0x7fdf639f8090>,
 <music21.stream.Score 0x7fdf63f66a10>]

In [103]:
#Helping function        
def extract_notes(file):
    notes = []
    pick = None
    for j in file:
        songs = instrument.partitionByInstrument(j)
        for part in songs.parts:
            pick = part.recurse()
            for element in pick:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append(".".join(str(n) for n in element.normalOrder))

    return notes
#Getting the list of notes as Corpus
Corpus= extract_notes(all_midis)
print("Total notes in all the Chopin midis in the dataset:", len(Corpus))

Total notes in all the Chopin midis in the dataset: 4586


In [104]:
Corpus

['C5',
 'C3',
 'E-4',
 'G3',
 'D4',
 'F3',
 'E-4',
 'G3',
 'C4',
 'E-3',
 'E-4',
 'G3',
 'D4',
 'F3',
 'E-4',
 'G3',
 'C5',
 'C3',
 'E-4',
 'G3',
 'D4',
 'F3',
 'E-4',
 'G3',
 'C4',
 'E-3',
 'E-4',
 'G3',
 'D4',
 'F3',
 'E-4',
 'G3',
 'G#4',
 'C3',
 'F4',
 'G#3',
 'E4',
 'G3',
 'F4',
 'G#3',
 'C4',
 'F3',
 'F4',
 'G#3',
 'E4',
 'G3',
 'F4',
 'G#3',
 'G#4',
 'C3',
 'F4',
 'G#3',
 'E4',
 'G3',
 'F4',
 'G#3',
 'C4',
 'F3',
 'F4',
 'G#3',
 'E4',
 'G3',
 'F4',
 'G#3',
 'B4',
 'C3',
 'F4',
 'G#3',
 'E-4',
 'G3',
 'F4',
 'G#3',
 'D4',
 'F3',
 'F4',
 'G#3',
 'E-4',
 'G3',
 'F4',
 'G#3',
 'B4',
 'C3',
 'F4',
 'G#3',
 'E-4',
 'G3',
 'F4',
 'G#3',
 'D4',
 'F3',
 'F4',
 'G#3',
 'E-4',
 'G3',
 'F4',
 'G#3',
 'C5',
 'C3',
 'G4',
 'E-3',
 'F4',
 'D3',
 'G4',
 'E-3',
 'E-4',
 'G3',
 'G4',
 'E-3',
 'F4',
 'D3',
 'G4',
 'E-3',
 'C5',
 'C3',
 'G4',
 'E-3',
 'F4',
 'D3',
 'G4',
 'E-3',
 'E-4',
 'G3',
 'G4',
 'E-3',
 'F4',
 'D3',
 'G4',
 'E-3',
 'E-5',
 'C3',
 'G#4',
 'C4',
 'G4',
 'B-3',
 'G#4',
 'C4',
 '

In [105]:
from fastai.text.all import *

In [106]:
text = ' . '.join([l.strip() for l in Corpus])
text[:100]

'C5 . C3 . E-4 . G3 . D4 . F3 . E-4 . G3 . C4 . E-3 . E-4 . G3 . D4 . F3 . E-4 . G3 . C5 . C3 . E-4 .'

In [113]:
text = text.replace('.','')

In [117]:
text

'C5  C3  E-4  G3  D4  F3  E-4  G3  C4  E-3  E-4  G3  D4  F3  E-4  G3  C5  C3  E-4  G3  D4  F3  E-4  G3  C4  E-3  E-4  G3  D4  F3  E-4  G3  G#4  C3  F4  G#3  E4  G3  F4  G#3  C4  F3  F4  G#3  E4  G3  F4  G#3  G#4  C3  F4  G#3  E4  G3  F4  G#3  C4  F3  F4  G#3  E4  G3  F4  G#3  B4  C3  F4  G#3  E-4  G3  F4  G#3  D4  F3  F4  G#3  E-4  G3  F4  G#3  B4  C3  F4  G#3  E-4  G3  F4  G#3  D4  F3  F4  G#3  E-4  G3  F4  G#3  C5  C3  G4  E-3  F4  D3  G4  E-3  E-4  G3  G4  E-3  F4  D3  G4  E-3  C5  C3  G4  E-3  F4  D3  G4  E-3  E-4  G3  G4  E-3  F4  D3  G4  E-3  E-5  C3  G#4  C4  G4  B-3  G#4  C4  E-4  G#3  G#4  C4  G4  B-3  G#4  C4  E-5  C3  G#4  C4  G4  B-3  G#4  C4  E-4  G#3  G#4  C4  G4  B-3  G#4  C4  D5  C3  F#4  A3  E4  G3  F#4  A3  D4  F#3  F#4  A3  E4  G3  F#4  A3  D5  C3  F#4  A3  E4  G3  F#4  A3  D4  F#3  F#4  A3  E4  G3  F#4  A3  D5  B-2  G4  B-3  F#4  A3  G4  B-3  D4  G3  G4  B-3  F#4  A3  G4  B-3  D5  B-2  G4  B-3  F#4  A3  G4  B-3  D4  G3  G4  B-3  F#4  A3  G4  B-3  C5  B-2  E4  G3  D4

In [114]:
tokens = text.split(' ')
tokens[:10]

['C5', '', 'C3', '', 'E-4', '', 'G3', '', 'D4', '']

In [115]:
vocab = L(*tokens).unique()

In [142]:
1/len(vocab)

0.013513513513513514

In [136]:
seql = 15

In [128]:
test_list =  tokens
odd_i = []
even_i = []
for i in range(0, len(test_list)):
    if i % 2:
        even_i.append(test_list[i])
    else :
        odd_i.append(test_list[i])
 
tokens = odd_i

In [129]:
word2idx = {w:i for i,w in enumerate(vocab)}
nums = L(word2idx[i] for i in tokens)
nums

(#4586) [0,2,3,4,5,6,3,4,7,8...]

In [152]:
L((tokens[i:i+3], tokens[i+3]) for i in range(0,len(tokens)-4,3))

(#1528) [(['C5', 'C3', 'E-4'], 'G3'),(['G3', 'D4', 'F3'], 'E-4'),(['E-4', 'G3', 'C4'], 'E-3'),(['E-3', 'E-4', 'G3'], 'D4'),(['D4', 'F3', 'E-4'], 'G3'),(['G3', 'C5', 'C3'], 'E-4'),(['E-4', 'G3', 'D4'], 'F3'),(['F3', 'E-4', 'G3'], 'C4'),(['C4', 'E-3', 'E-4'], 'G3'),(['G3', 'D4', 'F3'], 'E-4')...]

In [153]:
seqs = L((tensor(nums[i:i+3]), nums[i+3]) for i in range(0,len(nums)-4,3))
seqs

(#1528) [(tensor([0, 2, 3]), 4),(tensor([4, 5, 6]), 3),(tensor([3, 4, 7]), 8),(tensor([8, 3, 4]), 5),(tensor([5, 6, 3]), 4),(tensor([4, 0, 2]), 3),(tensor([3, 4, 5]), 6),(tensor([6, 3, 4]), 7),(tensor([7, 8, 3]), 4),(tensor([4, 5, 6]), 3)...]

In [154]:
bs = 32
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(seqs[:cut], seqs[cut:], bs=64, shuffle=False)

In [155]:
class LMModel2(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        
    def forward(self, x):
        h = 0
        for i in range(3):
            h = h + self.i_h(x[:,i])
            h = F.relu(self.h_h(h))
        return self.h_o(h)

In [156]:
learn = Learner(dls, LMModel2(len(vocab), 32), loss_func=F.cross_entropy, 
                metrics=accuracy)
learn.fit_one_cycle(4, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.372554,4.326988,0.003268,00:00
1,4.317542,4.259336,0.013072,00:00
2,4.257520,4.221972,0.019608,00:00
3,4.213894,4.214808,0.022876,00:00


In [157]:
class LMModel3(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        self.h = 0
        
    def forward(self, x):
        for i in range(3):
            self.h = self.h + self.i_h(x[:,i])
            self.h = F.relu(self.h_h(self.h))
        out = self.h_o(self.h)
        self.h = self.h.detach()
        return out
    
    def reset(self): self.h = 0

In [158]:
m = len(seqs)//bs
m,bs,len(seqs)

(47, 32, 1528)

In [159]:
def group_chunks(ds, bs):
    m = len(ds) // bs
    new_ds = L()
    for i in range(m): new_ds += L(ds[i + m*j] for j in range(bs))
    return new_ds

In [160]:
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(
    group_chunks(seqs[:cut], bs), 
    group_chunks(seqs[cut:], bs), 
    bs=bs, drop_last=True, shuffle=False)

In [161]:
learn = Learner(dls, LMModel3(len(vocab), 64), loss_func=F.cross_entropy,
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(10, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.249272,4.194565,0.031250,00:00
1,3.845809,3.828655,0.065972,00:00
2,3.455574,3.811248,0.055556,00:00
3,3.147003,3.846995,0.069444,00:00
4,2.887785,3.919418,0.076389,00:00
5,2.665069,4.023199,0.083333,00:00
6,2.476560,4.087889,0.079861,00:00
7,2.325493,4.077276,0.086806,00:00
8,2.217875,4.118322,0.097222,00:00
9,2.150937,4.124359,0.097222,00:00


In [162]:
sl = 15
seqs = L((tensor(nums[i:i+sl]), tensor(nums[i+1:i+sl+1]))
         for i in range(0,len(nums)-sl-1,sl))
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(group_chunks(seqs[:cut], bs),
                             group_chunks(seqs[cut:], bs),
                             bs=bs, drop_last=True, shuffle=False)

In [163]:
class LMModel4(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        self.h = 0
        
    def forward(self, x):
        outs = []
        for i in range(sl):
            self.h = self.h + self.i_h(x[:,i])
            self.h = F.relu(self.h_h(self.h))
            outs.append(self.h_o(self.h))
        self.h = self.h.detach()
        return torch.stack(outs, dim=1)
    
    def reset(self): self.h = 0

In [164]:
def loss_func(inp, targ):
    return F.cross_entropy(inp.view(-1, len(vocab)), targ.view(-1))

In [165]:
learn = Learner(dls, LMModel4(len(vocab), 64), loss_func=loss_func,
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(15, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.317391,4.329648,0.022917,00:00
1,4.271684,4.255292,0.037500,00:00
2,4.168822,4.127058,0.025000,00:00
3,3.989331,4.075876,0.012500,00:00
4,3.818932,4.094718,0.025000,00:00
5,3.675351,4.015654,0.029167,00:00
6,3.548419,4.020649,0.039583,00:00
7,3.436042,3.986885,0.045833,00:00
8,3.336870,3.978703,0.047917,00:00
9,3.248893,3.970929,0.045833,00:00


In [166]:
class LMModel5(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.RNN(n_hidden, n_hidden, n_layers, batch_first=True)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h = torch.zeros(n_layers, bs, n_hidden)
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(res)
    
    def reset(self): self.h.zero_()

In [167]:
learn = Learner(dls, LMModel5(len(vocab), 64, 2), 
                loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(15, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.293145,4.311300,0.010417,00:00
1,4.233229,4.224704,0.006250,00:00
2,4.071392,4.128365,0.006250,00:00
3,3.901986,4.111974,0.010417,00:00
4,3.768532,4.019567,0.012500,00:00
5,3.651724,3.955869,0.045833,00:00
6,3.543723,3.922219,0.050000,00:00
7,3.442343,3.909795,0.060417,00:00
8,3.349494,3.922462,0.070833,00:00
9,3.265605,3.895883,0.075000,00:00


In [168]:
class LMModel6(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.LSTM(n_hidden, n_hidden, n_layers, batch_first=True)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h = [torch.zeros(n_layers, bs, n_hidden) for _ in range(2)]
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = [h_.detach() for h_ in h]
        return self.h_o(res)
    
    def reset(self): 
        for h in self.h: h.zero_()

In [169]:
learn = Learner(dls, LMModel6(len(vocab), 64, 2), 
                loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(15, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.286817,4.259417,0.058333,00:00
1,4.177755,4.072579,0.050000,00:00
2,3.955873,4.162927,0.035417,00:00
3,3.819813,4.017930,0.033333,00:00
4,3.715737,3.983724,0.029167,00:00
5,3.624076,3.953107,0.031250,00:00
6,3.537251,3.866549,0.054167,00:00
7,3.454556,3.803925,0.058333,00:00
8,3.375802,3.857017,0.058333,00:00
9,3.302118,3.835922,0.060417,00:00


In [170]:
class Dropout(Module):
    def __init__(self, p): self.p = p
    def forward(self, x):
        if not self.training: return x
        mask = x.new(*x.shape).bernoulli_(1-p)
        return x * mask.div_(1-p)

In [171]:
class LMModel7(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers, p):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.LSTM(n_hidden, n_hidden, n_layers, batch_first=True)
        self.drop = nn.Dropout(p)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h_o.weight = self.i_h.weight
        self.h = [torch.zeros(n_layers, bs, n_hidden) for _ in range(2)]
        
    def forward(self, x):
        raw,h = self.rnn(self.i_h(x), self.h)
        out = self.drop(raw)
        self.h = [h_.detach() for h_ in h]
        return self.h_o(out),raw,out
    
    def reset(self): 
        for h in self.h: h.zero_()

In [172]:
learn = TextLearner(dls, LMModel7(len(vocab), 64, 2, 0.4),
                    loss_func=CrossEntropyLossFlat(), metrics=accuracy)

In [173]:
learn.fit_one_cycle(15, 1e-2, wd=0.1)

epoch,train_loss,valid_loss,accuracy,time
0,4.414560,4.241100,0.027083,00:00
1,4.198083,4.123768,0.035417,00:00
2,3.990653,3.934480,0.068750,00:00
3,3.800861,3.866560,0.093750,00:00
4,3.619548,3.799883,0.081250,00:00
5,3.446004,3.815181,0.095833,00:00
6,3.281458,3.832448,0.081250,00:00
7,3.117347,3.891773,0.075000,00:00
8,2.973493,3.886087,0.083333,00:00
9,2.834132,4.015900,0.072917,00:00
